In [2]:
#from data_science_toolkit.scraping.scraping import *
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException

# search each link 
import requests 
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import math
import re
import time
import os
# lets open booking:

dfolder='/home/pere/ITMNLP_DSDM/booking'
geko_path='/home/pere/ITMNLP_DSDM/geckodriver-v0.34.0-linux64/geckodriver'
link='https://www.booking.com/index.en.html'


# browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path)



In [48]:
def ffx_preferences(dfolder, download=False):
    '''
    Sets the preferences of the firefox browser: download path.
    '''
    profile = webdriver.FirefoxProfile()
    # set download folder:
    profile.set_preference("browser.download.dir", dfolder)
    profile.set_preference("browser.download.folderList", 2)
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk",
                           "application/msword,application/rtf, application/csv,text/csv,image/png ,image/jpeg, application/pdf, text/html,text/plain,application/octet-stream")
    
    #profile.add_extension('/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse/booking/booking/ublock_origin-1.55.0.xpi')


    # this allows to download pdfs automatically
    if download:
        profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf,application/x-pdf")
        profile.set_preference("pdfjs.disabled", True)

    options = Options()
    options.profile = profile
    return options


def start_up(link, dfolder, geko_path,donwload=True):
    # geko_path='/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse/booking/geckodriver'
    # download_path='./downloads'
    os.makedirs(dfolder, exist_ok=True)

    options = ffx_preferences(dfolder,donwload)
    service = Service(geko_path)
    browser = webdriver.Firefox(service=service, options=options)
    # Enter the website address here
    browser.get(link)
    time.sleep(5)  # Adjust sleep time as needed
    return browser


def check_and_click(browser, xpath, type):
    '''
    Function that checks whether the object is clickable and, if so, clicks on
    it. If not, waits one second and tries again.
    '''
    ck = False
    ss = 0
    while ck == False:
        ck = check_obscures(browser, xpath, type)
        time.sleep(1)
        ss += 1
        if ss == 15:
            # warn_sound()
            # return NoSuchElementException
            ck = True
            # browser.quit()

def check_obscures(browser, xpath, type):
    '''
    Function that checks whether the object is being "obscured" by any element so
    that it is not clickable. Important: if True, the object is going to be clicked!
    '''
    try:
        if type == "xpath":
            browser.find_element('xpath',xpath).click()
        elif type == "id":
            browser.find_element('id',xpath).click()
        elif type == "css":
            browser.find_element('css selector',xpath).click()
        elif type == "class":
            browser.find_element('class name',xpath).click()
        elif type == "link":
            browser.find_element('link text',xpath).click()
    except (ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException) as e:
        print(e)
        return False
    return True


link='https://www.booking.com'
firefox_binary_path = r'C:/Program Files/Mozilla Firefox/firefox.exe'  # Example path, adjust as needed


browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path)

In [32]:
def get_number_pages(browser):
    '''
    Get the number of pages. 
    '''
    a = browser.find_elements('xpath','//div[@class="d8f77e681c"]')
    #number of total hotel listings divided of listings p. page
    total_listings = int(re.findall(r'\d+', a[-1].text)[0])
    page_num = math.floor(total_listings / 25)
    print(f'Total hotels: {total_listings} \nNumber of pages: {page_num}')
    return(page_num) 

In [33]:
def search_hotels(start_date, end_date, place, browser):
    browser.get('https://www.booking.com')
    # Clear browsing history and cookies
    browser.delete_all_cookies()
    xpath = '//*[@id="onetrust-accept-btn-handler"]'

    time.sleep(10)
    try:
        check_and_click(browser, xpath, type='xpath')
    except:
        pass
    time.sleep(5)

    #accept cookies, refresh page then exit pop up (if it appears)
    browser.refresh()
    browser.refresh()
    browser.refresh()
    time.sleep(10)
    try:
        check_and_click(browser, '//button[@aria-label="Dismiss sign-in info."]', type='xpath')
    except:
    # Handle the exception without displaying the error message
        pass
    
    try:
        # Switch to the iframe and exit the google long in pop up
        iframe_path  = (By.XPATH, '//iframe[@title="Cuadro de diálogo Iniciar sesión con Google"]')
        iframe = WebDriverWait(browser, 10).until(EC.frame_to_be_available_and_switch_to_it(iframe_path))

        element_locator = (By.XPATH, '//div[@aria-label="Cerrar"]')
        element = WebDriverWait(browser, 10).until(EC.element_to_be_clickable(element_locator))
        element.click()

        browser.switch_to.default_content()
    except Exception as e:
        pass

        
    search1 = browser.find_element(by='xpath',value='//*[@id=":re:"]')
    search1.send_keys(place)

    #click date selector

    css='button.ebbedaf8ac:nth-child(2) > span:nth-child(1)'

    browser.find_element('css selector',css).click()


    #click start and end date

    start_date_found = False
    end_date_found = False
    while start_date_found == False or end_date_found == False:
        path='//span[@class="cf06f772fa"]'
        dates = browser.find_elements('xpath',path)
        for date in dates:
            if date.get_attribute("data-date") == start_date:
                date.click()
                start_date_found = True
            if date.get_attribute("data-date") == end_date:
                date.click()
                end_date_found = True
                break
        next_page_button = '//button[@class="a83ed08757 c21c56c305 f38b6daa18 d691166b09 f671049264 deab83296e f4552b6561 dc72a8413c f073249358"]'
        check_and_click(browser, next_page_button, type='xpath')
        time.sleep(3)

    time.sleep(3)

    #click search button
    my_xpath='/html/body/div[3]/div[2]/div/form/div[1]/div[4]/button'

    check_obscures(browser,my_xpath , type='xpath')
    check_and_click(browser,my_xpath , type='xpath')

    hotel_box_path = '//div[@data-testid="filters-group-label-content" and contains(text(), "Hotels")]'
    # hotel_box_path = '/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[1]/div[3]/div[4]/div[8]/label/span[3]/div/div'
    hotel_boxes = browser.find_elements('xpath', hotel_box_path)

    if hotel_boxes:
        hotel_boxes[0].click()
    else:
        print("Element not found")
    time.sleep(5)

    # # Change language to english
    # lang_button = 'div.aca0ade214:nth-child(2) > span:nth-child(2) > button:nth-child(1)'
    # eng_button = 'div.ebac6e22e9 > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > ul:nth-child(1) > li:nth-child(3) > button:nth-child(1)'
    # browser.find_element('css selector',lang_button).click()
    # time.sleep(2)
    # browser.find_element('css selector',eng_button).click()
    # time.sleep(5)  # wait for language change to take effect

    pages = get_number_pages(browser)
    print(pages)

    hotel_card_container = '//div[@data-testid="property-card"]'  # XPath expression to locate the hotel card container

    #check "Hotels" box

    

        


    data = []  # List to store the extracted data for each hotel


    for page in range(pages+1):
        hotels = browser.find_elements('xpath', hotel_card_container)  # Find all hotel elements on each iteration
        for hotel in hotels:  # Iterate over each hotel element
            data_dic = {}  # Dictionary to store the extracted data for the current hotel

            try:
                # Extract the photo URL of the hotel
                photo_path = './/img[@data-testid="image"]'  # XPath expression to locate the photo element
                photo_url = hotel.find_element('xpath', photo_path).get_attribute('src')  # Get the src attribute of the photo element
                data_dic['photo_url'] = photo_url  # Store the photo URL in the dictionary
            except NoSuchElementException:
                print('No photo URL')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the link of the hotel
                link_path = './/a[@class="a78ca197d0"]'  # XPath expression to locate the link element
                link = hotel.find_element('xpath', link_path).get_attribute('href')  # Get the href attribute of the link element
                data_dic['link'] = link  # Store the link in the dictionary
            except NoSuchElementException:
                print('No link')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the name of the hotel
                name_path = './/div[@data-testid="title"]'  # XPath expression to locate the name element
                name = hotel.find_element('xpath', name_path).text  # Get the text content of the name element
                data_dic['name'] = name  # Store the name in the dictionary
            except NoSuchElementException:
                print('No name')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the central distance of the hotel
                central_distance_path = './/span[@data-testid="distance"]'  # XPath expression to locate the central distance element
                central_distance = hotel.find_element('xpath', central_distance_path).text  # Get the text content of the central distance element
                data_dic['central_distance'] = central_distance  # Store the central distance in the dictionary
            except NoSuchElementException:
                print('No central distance')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the sustainability information of the hotel
                sustainability_path = './/span[@class="abf093bdfe d068504c75 f68ecd98ea"]'  # XPath expression to locate the sustainability element
                sustainability = hotel.find_element('xpath', sustainability_path).text  # Get the text content of the sustainability element
                data_dic['sustainability'] = sustainability  # Store the sustainability information in the dictionary
            except NoSuchElementException:
                data_dic['sustainability'] = None
                print('No sustainability information')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the rating of the hotel
                rating_path = './/div[@class="a3b8729ab1 d86cee9b25"]'  # XPath expression to locate the rating element
                rating = hotel.find_element('xpath', rating_path).text  # Get the text content of the rating element
                data_dic['rating'] = rating  # Store the rating in the dictionary
            except NoSuchElementException:
                print('No rating')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the comfort rating of the hotel
                comfort_rating_path = './/span[@class="a3332d346a"]'  # XPath expression to locate the comfort rating element
                comfort_rating = hotel.find_element('xpath', comfort_rating_path).text  # Get the text content of the comfort rating element
                data_dic['comfort_rating'] = comfort_rating  # Store the comfort rating in the dictionary
            except NoSuchElementException:
                print('No comfort rating')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the review count of the hotel
                review_count_path = './/div[@class="abf093bdfe f45d8e4c32 d935416c47"]'  # XPath expression to locate the review count element
                review_count = hotel.find_element('xpath', review_count_path).text  # Get the text content of the review count element
                data_dic['review_count'] = review_count  # Store the review count in the dictionary
            except NoSuchElementException:
                print('No review count')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the room description of the hotel
                room_description_path = './/h4[@class="abf093bdfe e8f7c070a7"]'  # XPath expression to locate the room description element
                room_description = hotel.find_element('xpath', room_description_path).text  # Get the text content of the room description element
                data_dic['room_description'] = room_description  # Store the room description in the dictionary
            except NoSuchElementException:
                print('No room description')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the room specifics of the hotel
                room_specifics_path = './/div[@class="abf093bdfe"]'  # XPath expression to locate the room specifics elements
                room_specifics = hotel.find_elements('xpath', room_specifics_path)  # Find all room specifics elements
                room_specifics_text = []
                for room_specific in room_specifics:
                    room_specifics_text.append(room_specific.text)  # Get the text content of each room specifics element
                data_dic['room_specifics'] = ' '.join(room_specifics_text)  # Store the room specifics as a single string in the dictionary
            except NoSuchElementException:
                print('No room specifics')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the price of the hotel
                price_path = './/span[@data-testid="price-and-discounted-price"]'  # XPath expression to locate the price element
                price = hotel.find_element('xpath', price_path).text  # Get the text content of the price element
                data_dic['price'] = price  # Store the price in the dictionary
            except NoSuchElementException:
                print('No price')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the original price of the hotel (if available)
                original_price_path = './/span[@class="c73ff05531 e84eb96b1f"]'  # XPath expression to locate the original price element
                original_price = hotel.find_element('xpath', original_price_path).text  # Get the text content of the original price element
                data_dic['original_price'] = original_price  # Store the original price in the dictionary
            except NoSuchElementException:
                data_dic['original_price'] = None  # If the original price element is not found, store None in the dictionary
                print('No original price')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the neighborhood and city of the hotel
                neighborhood_city_path = './/span[@data-testid="address"]'  # XPath expression to locate the neighborhood and city element
                neighborhood_city = hotel.find_element('xpath', neighborhood_city_path).text  # Get the text content of the neighborhood and city element
                data_dic['neighborhood_city'] = neighborhood_city  # Store the neighborhood and city in the dictionary
            except NoSuchElementException:
                print('No neighborhood and city')
            except Exception as e:
                print(e)
                print('Something went wrong')

            data.append(data_dic)  # Add the dictionary to the data list
            data_dic = {}
        # The data list now contains the extracted data for all hotels
        next_page_path = '//button[@aria-label="Next page"]'
        next_page = browser.find_elements('xpath', next_page_path)
        print(next_page)
        #element = WebDriverWait(browser, 30).until(EC.element_to_be_clickable(next_page[0]))
        next_page[0].click()
        time.sleep(3)
    data = pd.DataFrame(data)
    data['start_date'] = start_date
    data['end_date'] = end_date
    return data
        

In [34]:
def get_hotel_details(data):
    hotels_data = []
    x = 0 #just to tell how far the process is
    for link in data.link:
        hotel_data = {}
        print(link)
        x += 1
        print(f'{x} out of {len(data)}')
        r = requests.get(link)
        soup = BeautifulSoup(r.content, 'html.parser')
        id="hotel_sidebar_static_map"

        # map_element = soup.find('a', id='hotel_sidebar_static_map').get('data-atlas-latlng')
         #hotel_data['map_lat'] = map_element.split(',')[0]
        # hotel_data['map_lng'] = map_element.split(',')[1]

        hotel_data['name'] = soup.find('h2', class_='d2fee87262 pp-header__title').get_text()
        #hotel_data['address'] = soup.find('span', class_='hp_address_subtitle').get_text()
        hotel_data['description'] = soup.find('div', class_='hp-description').get_text()

        # room_services = soup.find_all('span', class_='a5a5a75131')
        # hotel_data['room_services'] = ', '.join([room_service.get_text() for room_service in room_services])
        
        # room_ratings = soup.find_all('div', class_='ccb65902b2')
        # room_ratings_desc = [element.get_text() for element in room_ratings if ',' not in element.get_text()]
        # room_ratings = [element.get_text() for element in room_ratings if ',' in element.get_text()]


        # for index in range(0,len(room_ratings)): 
        #     hotel_data[f'room_rating_{room_ratings_desc[index]}'] = room_ratings[index]

    
        #hotel_data['room_options_count'] = len(soup.find_all('tr', class_='js-rt-block-row'))

        hotels_data.append(hotel_data)
    hotels_data = pd.DataFrame(hotels_data)
    return pd.concat([data, hotels_data], axis=1)


In [21]:
start_date = '2024-06-17'
end_date = '2024-06-23'
place = "Barcelona"
file_path = start_date +'_'+end_date+'_'+place+'.csv'
data_c1_w1 = search_hotels(browser=browser, start_date=start_date,end_date=end_date,place=place)
data_c1_w1 = get_hotel_details(data_c1_w1)
data_c1_w1.to_csv(file_path)

Message: Unable to locate element: //button[@aria-label="Dismiss sign-in info."]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

Message: Unable to locate element: //button[@aria-label="Dismiss sign-in info."]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find

Message: Unable to locate element: /html/body/div[3]/div[2]/div/form/div[1]/div[4]/button; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

Message: Unable to locate element: /html/body/div[3]/div[2]/div/form/div[1]/div[4]/button; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.

No original price
No original price
No comfort rating
No original price
No sustainability information
No original price
No comfort rating
No original price
[<selenium.webdriver.remote.webelement.WebElement (session="0eff8fcc-e6dd-47b8-8755-025447674929", element="a661d82c-b866-44bc-bc65-ed8f2e87cf91")>]
No sustainability information
No comfort rating
No original price
No comfort rating
No original price
No comfort rating
No original price
No original price
No original price
No original price
No original price
No original price
No comfort rating
No original price
No original price
No comfort rating
No comfort rating
No original price
No original price
No comfort rating
No original price
No original price
No original price
No comfort rating
No original price
No comfort rating
No original price
No sustainability information
No comfort rating
No original price
No comfort rating
No original price
No original price
[<selenium.webdriver.remote.webelement.WebElement (session="0eff8fcc-e6dd-47b

No comfort rating
No original price
No comfort rating
No original price
No sustainability information
No comfort rating
No comfort rating
No original price
No comfort rating
No original price
No sustainability information
No original price
No comfort rating
No original price
No comfort rating
No sustainability information
No sustainability information
No comfort rating
No comfort rating
No original price
[<selenium.webdriver.remote.webelement.WebElement (session="0eff8fcc-e6dd-47b8-8755-025447674929", element="a661d82c-b866-44bc-bc65-ed8f2e87cf91")>]
No comfort rating
No original price
No comfort rating
No original price
No original price
No sustainability information
No original price
No sustainability information
No sustainability information
No original price
No comfort rating
No original price
No original price
No sustainability information
No comfort rating
No original price
No comfort rating
No original price
No comfort rating
No original price
No comfort rating
No original price

https://www.booking.com/hotel/es/two-barcelona-by-axel.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4AtfY6a0GwAIB0gIkNDU5ZTg0N2ItOTNiOC00NmM2LWEwYWMtMzdjNzBhZjZkNjZk2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-17&checkout=2024-06-23&dest_id=-372490&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=3&hapos=3&sr_order=popularity&nflt=ht_id%3D204&srpvid=81d76f8970f4002f&srepoch=1706716328&all_sr_blocks=131688502_0_2_0_0&highlighted_blocks=131688502_0_2_0_0&matching_block_id=131688502_0_2_0_0&sr_pri_blocks=131688502_0_2_0_0__124741&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
3 out of 378
https://www.booking.com/hotel/es/spa-villa-olimpic-suites.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4AtfY6a0GwAIB0gIkNDU5ZTg0N2ItOTNiOC00NmM2LWEwYWMtMzdjNzBhZjZkNjZk2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-17&checkout=2024-06-23&dest_id=-372490&dest_type=city&group_a

...

https://www.booking.com/hotel/es/duquesadecardona.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4AtfY6a0GwAIB0gIkNDU5ZTg0N2ItOTNiOC00NmM2LWEwYWMtMzdjNzBhZjZkNjZk2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-17&checkout=2024-06-23&dest_id=-372490&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=25&hapos=275&sr_order=popularity&nflt=ht_id%3D204&srpvid=81d76f8970f4002f&srepoch=1706716390&all_sr_blocks=9063601_257911618_0_2_0&highlighted_blocks=9063601_257911618_0_2_0&matching_block_id=9063601_257911618_0_2_0&sr_pri_blocks=9063601_257911618_0_2_0__239380&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
303 out of 378
https://www.booking.com/hotel/es/duquesadecardona.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4AtfY6a0GwAIB0gIkNDU5ZTg0N2ItOTNiOC00NmM2LWEwYWMtMzdjNzBhZjZkNjZk2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-17&checkout=2024-06-23&dest_id=-372490&dest_t

https://www.booking.com/hotel/es/del-mar.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4AtfY6a0GwAIB0gIkNDU5ZTg0N2ItOTNiOC00NmM2LWEwYWMtMzdjNzBhZjZkNjZk2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-17&checkout=2024-06-23&dest_id=-372490&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=12&hapos=287&sr_order=popularity&nflt=ht_id%3D204&srpvid=81d76f8970f4002f&srepoch=1706716396&all_sr_blocks=9254701_88192667_2_2_0&highlighted_blocks=9254701_88192667_2_2_0&matching_block_id=9254701_88192667_2_2_0&sr_pri_blocks=9254701_88192667_2_2_0__131436&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
315 out of 378
https://www.booking.com/hotel/es/astonbarcelona.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4AtfY6a0GwAIB0gIkNDU5ZTg0N2ItOTNiOC00NmM2LWEwYWMtMzdjNzBhZjZkNjZk2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-17&checkout=2024-06-23&dest_id=-372490&dest_type=city&group_

https://www.booking.com/hotel/es/lalguer.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4AtfY6a0GwAIB0gIkNDU5ZTg0N2ItOTNiOC00NmM2LWEwYWMtMzdjNzBhZjZkNjZk2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-17&checkout=2024-06-23&dest_id=-372490&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=24&hapos=299&sr_order=popularity&nflt=ht_id%3D204&srpvid=81d76f8970f4002f&srepoch=1706716396&all_sr_blocks=9039908_91908082_0_0_0&highlighted_blocks=9039908_91908082_0_0_0&matching_block_id=9039908_91908082_0_0_0&sr_pri_blocks=9039908_91908082_0_0_0__96556&from=searchresults#hotelTmpl
327 out of 378
https://www.booking.com/hotel/es/aparthotel-silver.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4AtfY6a0GwAIB0gIkNDU5ZTg0N2ItOTNiOC00NmM2LWEwYWMtMzdjNzBhZjZkNjZk2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-17&checkout=2024-06-23&dest_id=-372490&dest_type=city&group_adults=2&req_adults=2&no_room

https://www.booking.com/hotel/es/libere-barcelona-sant-antoni.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4AtfY6a0GwAIB0gIkNDU5ZTg0N2ItOTNiOC00NmM2LWEwYWMtMzdjNzBhZjZkNjZk2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-17&checkout=2024-06-23&dest_id=-372490&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=11&hapos=311&sr_order=popularity&nflt=ht_id%3D204&srpvid=81d76f8970f4002f&srepoch=1706716402&all_sr_blocks=9179719_204775730_0_0_0&highlighted_blocks=9179719_204775730_0_0_0&matching_block_id=9179719_204775730_0_0_0&sr_pri_blocks=9179719_204775730_0_0_0__157494&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
339 out of 378
https://www.booking.com/hotel/es/hotelpulitzer.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4AtfY6a0GwAIB0gIkNDU5ZTg0N2ItOTNiOC00NmM2LWEwYWMtMzdjNzBhZjZkNjZk2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-17&checkout=2024-06-23&dest_id=-3724

AttributeError: 'NoneType' object has no attribute 'get'

In [35]:
start_date = '2024-06-03'
end_date = '2024-06-09'
place = "Barcelona"
file_path = start_date +'_'+end_date+'_'+place+'.csv'
data_c1_w2 = search_hotels(browser=browser, start_date=start_date,end_date=end_date,place=place)
data_c1_w2 = get_hotel_details(data_c1_w2)

Message: Unable to locate element: //button[@aria-label="Dismiss sign-in info."]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

Message: Unable to locate element: //button[@aria-label="Dismiss sign-in info."]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find

Message: Unable to locate element: /html/body/div[3]/div[2]/div/form/div[1]/div[4]/button; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

Message: Unable to locate element: /html/body/div[3]/div[2]/div/form/div[1]/div[4]/button; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.

No original price
No sustainability information
No comfort rating
No original price
No comfort rating
No original price
No original price
No comfort rating
No original price
No sustainability information
No original price
No comfort rating
No original price
[<selenium.webdriver.remote.webelement.WebElement (session="1137cb86-cfd5-469f-935d-4c34aa925665", element="938bad41-63c0-4ed9-992a-3b0acce79a60")>]
No comfort rating
No original price
No comfort rating
No original price
No sustainability information
No comfort rating
No original price
No comfort rating
No original price
No comfort rating
No original price
No comfort rating
No original price
No comfort rating
No original price
No comfort rating
No original price
No comfort rating
No original price
No comfort rating
No original price
No sustainability information
No original price
No comfort rating
No original price
No comfort rating
No original price
No original price
No original price
No comfort rating
No original price
No original

No comfort rating
No original price
No comfort rating
No sustainability information
No comfort rating
No original price
No comfort rating
No original price
No comfort rating
No original price
No sustainability information
No comfort rating
No sustainability information
No comfort rating
No comfort rating
No original price
No sustainability information
No comfort rating
No comfort rating
No original price
No comfort rating
No original price
No sustainability information
No comfort rating
No original price
No sustainability information
No sustainability information
No comfort rating
No original price
No sustainability information
No comfort rating
No comfort rating
No original price
No original price
No comfort rating
No original price
[<selenium.webdriver.remote.webelement.WebElement (session="1137cb86-cfd5-469f-935d-4c34aa925665", element="938bad41-63c0-4ed9-992a-3b0acce79a60")>]
No comfort rating
No original price
No original price
No original price
No original price
No original price

https://www.booking.com/hotel/es/sansi-barcelona-barcelona.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4AoWy7a0GwAIB0gIkYTEyZTE0MmUtNjQzMy00NGRjLWI4NDctMDcyYWRlMjA3OGYw2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&dest_id=-372490&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=3&hapos=3&sr_order=popularity&nflt=ht_id%3D204&srpvid=551f3d1d747e00f1&srepoch=1706776913&all_sr_blocks=0_0_2_0_0&highlighted_blocks=0_0_2_0_0&matching_block_id=0_0_2_0_0&sr_pri_blocks=0_0_2_0_0__103406&from=searchresults#hotelTmpl
3 out of 437
https://www.booking.com/hotel/es/hotelalimara.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4AoWy7a0GwAIB0gIkYTEyZTE0MmUtNjQzMy00NGRjLWI4NDctMDcyYWRlMjA3OGYw2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&dest_id=-372490&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=4&

...

https://www.booking.com/hotel/es/toledano-ramblas.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4AoWy7a0GwAIB0gIkYTEyZTE0MmUtNjQzMy00NGRjLWI4NDctMDcyYWRlMjA3OGYw2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&dest_id=-372490&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=10&hapos=385&sr_order=popularity&nflt=ht_id%3D204&srpvid=551f3d1d747e00f1&srepoch=1706777003&all_sr_blocks=48271301_88209831_0_0_0&highlighted_blocks=48271301_88209831_0_0_0&matching_block_id=48271301_88209831_0_0_0&sr_pri_blocks=48271301_88209831_0_0_0__104406&from=searchresults#hotelTmpl
387 out of 437
https://www.booking.com/hotel/es/alfaaeropuerto.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4AoWy7a0GwAIB0gIkYTEyZTE0MmUtNjQzMy00NGRjLWI4NDctMDcyYWRlMjA3OGYw2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&dest_id=-372490&dest_type=city&group_adults=2&req_adult

https://www.booking.com/hotel/es/gotico.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4AoWy7a0GwAIB0gIkYTEyZTE0MmUtNjQzMy00NGRjLWI4NDctMDcyYWRlMjA3OGYw2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&dest_id=-372490&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=22&hapos=397&sr_order=popularity&nflt=ht_id%3D204&srpvid=551f3d1d747e00f1&srepoch=1706777003&all_sr_blocks=9265501_88192811_2_2_0&highlighted_blocks=9265501_88192811_2_2_0&matching_block_id=9265501_88192811_2_2_0&sr_pri_blocks=9265501_88192811_2_2_0__150774&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
399 out of 437
https://www.booking.com/hotel/es/sonder-paseo-de-gracia.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4AoWy7a0GwAIB0gIkYTEyZTE0MmUtNjQzMy00NGRjLWI4NDctMDcyYWRlMjA3OGYw2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&dest_id=-372490&dest_type=city

https://www.booking.com/hotel/es/nh-constanza.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4AoWy7a0GwAIB0gIkYTEyZTE0MmUtNjQzMy00NGRjLWI4NDctMDcyYWRlMjA3OGYw2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&dest_id=-372490&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=9&hapos=409&sr_order=popularity&nflt=ht_id%3D204&srpvid=551f3d1d747e00f1&srepoch=1706777009&all_sr_blocks=9385421_93709751_0_2_0&highlighted_blocks=9385421_93709751_0_2_0&matching_block_id=9385421_93709751_0_2_0&sr_pri_blocks=9385421_93709751_0_2_0__178480&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
411 out of 437
https://www.booking.com/hotel/es/sonder-los-arcos.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4AoWy7a0GwAIB0gIkYTEyZTE0MmUtNjQzMy00NGRjLWI4NDctMDcyYWRlMjA3OGYw2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&dest_id=-372490&dest_type=city&

https://www.booking.com/hotel/es/hoteladvancebarcelona.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4AoWy7a0GwAIB0gIkYTEyZTE0MmUtNjQzMy00NGRjLWI4NDctMDcyYWRlMjA3OGYw2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&dest_id=-372490&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=21&hapos=421&sr_order=popularity&nflt=ht_id%3D204&srpvid=551f3d1d747e00f1&srepoch=1706777009&all_sr_blocks=9133609_88190977_2_2_0&highlighted_blocks=9133609_88190977_2_2_0&matching_block_id=9133609_88190977_2_2_0&sr_pri_blocks=9133609_88190977_2_2_0__152427&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
423 out of 437
https://www.booking.com/hotel/es/ofelias-4-superior.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4AoWy7a0GwAIB0gIkYTEyZTE0MmUtNjQzMy00NGRjLWI4NDctMDcyYWRlMjA3OGYw2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&dest_id=-372490&des

https://www.booking.com/hotel/es/linda-habitacion-privada-barcelona.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4AoWy7a0GwAIB0gIkYTEyZTE0MmUtNjQzMy00NGRjLWI4NDctMDcyYWRlMjA3OGYw2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&dest_id=-372490&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=8&hapos=433&sr_order=popularity&nflt=ht_id%3D204&srpvid=551f3d1d747e00f1&srepoch=1706777014&all_sr_blocks=1120781601_384876749_2_0_0&highlighted_blocks=1120781601_384876749_2_0_0&matching_block_id=1120781601_384876749_2_0_0&sr_pri_blocks=1120781601_384876749_2_0_0__157400&from=searchresults#hotelTmpl
435 out of 437
https://www.booking.com/hotel/es/beach-style-loft-5-min-from-the-beach.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4AoWy7a0GwAIB0gIkYTEyZTE0MmUtNjQzMy00NGRjLWI4NDctMDcyYWRlMjA3OGYw2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&des

In [47]:
start_date = '2024-06-17'
end_date = '2024-06-23'
place = "Madrid, Spain"
file_path = start_date +'_'+end_date+'_'+place+'.csv'
data_c2_w1 = search_hotels(browser=browser, start_date=start_date,end_date=end_date,place=place)
data_c2_w1 = get_hotel_details(data_c2_w1)

Message: Unable to locate element: //button[@aria-label="Dismiss sign-in info."]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:509:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Message: Unable to locate element: //button[@aria-label="Dismiss sign-in info."]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:509:5
dom.find

Message: Unable to locate element: /html/body/div[3]/div[2]/div/form/div[1]/div[4]/button; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:509:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Message: Unable to locate element: /html/body/div[3]/div[2]/div/form/div[1]/div[4]/button; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.

No original price
No sustainability information
No comfort rating
No sustainability information
No original price
No original price
[<selenium.webdriver.remote.webelement.WebElement (session="6523097c-02f4-4b6a-8819-2d6947a9474d", element="09f853e3-415f-4a8e-9811-40219de7e4d5")>]
No original price
No original price
No original price
No sustainability information
No original price
No comfort rating
No original price
No comfort rating
No original price
No comfort rating
No original price
No original price
No sustainability information
No original price
No comfort rating
No original price
No comfort rating
No original price
No comfort rating
No original price
No comfort rating
No original price
No comfort rating
No original price
No sustainability information
No comfort rating
No original price
No original price
No sustainability information
No comfort rating
No comfort rating
No original price
No sustainability information
No original price
No comfort rating
No original price
No comfort 

No original price
No comfort rating
No original price
[<selenium.webdriver.remote.webelement.WebElement (session="6523097c-02f4-4b6a-8819-2d6947a9474d", element="09f853e3-415f-4a8e-9811-40219de7e4d5")>]
No comfort rating
No original price
No comfort rating
No original price
No original price
No original price
No original price
No original price
No comfort rating
No original price
No comfort rating
No original price
No sustainability information
No comfort rating
No original price
No comfort rating
No original price
No comfort rating
No original price
No comfort rating
No original price
No original price
No original price
No sustainability information
No comfort rating
No original price
No comfort rating
No original price
No comfort rating
No original price
No original price
No sustainability information
No comfort rating
No original price
No sustainability information
No comfort rating
No original price
No comfort rating
No original price
No original price
No sustainability information

https://www.booking.com/hotel/es/apartamentos-juan-bravo.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Aqvp7a0GwAIB0gIkMTIwY2VhY2QtNzJlMi00Yjg2LWJmYmEtMGZiNTI3YTc4NmVi2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-17&checkout=2024-06-23&dest_id=-390625&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=3&hapos=3&sr_order=popularity&nflt=ht_id%3D204&srpvid=273d4af163be01eb&srepoch=1706783992&all_sr_blocks=1241005_354411197_2_0_0&highlighted_blocks=1241005_354411197_2_0_0&matching_block_id=1241005_354411197_2_0_0&sr_pri_blocks=1241005_354411197_2_0_0__74875&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
3 out of 353
https://www.booking.com/hotel/es/axel-madrid-madrid12345678.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Aqvp7a0GwAIB0gIkMTIwY2VhY2QtNzJlMi00Yjg2LWJmYmEtMGZiNTI3YTc4NmVi2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-17&checkout=2024-06-23&dest_id=-39

...

https://www.booking.com/hotel/es/tach-by-clementhoteles.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Aqvp7a0GwAIB0gIkMTIwY2VhY2QtNzJlMi00Yjg2LWJmYmEtMGZiNTI3YTc4NmVi2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-17&checkout=2024-06-23&dest_id=-390625&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=2&hapos=302&sr_order=popularity&nflt=ht_id%3D204&srpvid=273d4af163be01eb&srepoch=1706784079&all_sr_blocks=1846902_95143297_0_2_0&highlighted_blocks=1846902_95143297_0_2_0&matching_block_id=1846902_95143297_0_2_0&sr_pri_blocks=1846902_95143297_0_2_0__102000&from=searchresults#hotelTmpl
303 out of 353
https://www.booking.com/hotel/es/avenue.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Aqvp7a0GwAIB0gIkMTIwY2VhY2QtNzJlMi00Yjg2LWJmYmEtMGZiNTI3YTc4NmVi2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-17&checkout=2024-06-23&dest_id=-390625&dest_type=city&group_adults=2&req_adults=2&no_

https://www.booking.com/hotel/es/alexandra-madrid.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Aqvp7a0GwAIB0gIkMTIwY2VhY2QtNzJlMi00Yjg2LWJmYmEtMGZiNTI3YTc4NmVi2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-17&checkout=2024-06-23&dest_id=-390625&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=14&hapos=314&sr_order=popularity&nflt=ht_id%3D204&srpvid=273d4af163be01eb&srepoch=1706784079&all_sr_blocks=2799501_95152000_0_2_0&highlighted_blocks=2799501_95152000_0_2_0&matching_block_id=2799501_95152000_0_2_0&sr_pri_blocks=2799501_95152000_0_2_0__163350&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
315 out of 353
https://www.booking.com/hotel/es/air-rooms-madrid.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Aqvp7a0GwAIB0gIkMTIwY2VhY2QtNzJlMi00Yjg2LWJmYmEtMGZiNTI3YTc4NmVi2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-17&checkout=2024-06-23&dest_id=-390625&dest_type=

https://www.booking.com/hotel/es/rio-by-wynwood-house.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Aqvp7a0GwAIB0gIkMTIwY2VhY2QtNzJlMi00Yjg2LWJmYmEtMGZiNTI3YTc4NmVi2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-17&checkout=2024-06-23&dest_id=-390625&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=326&sr_order=popularity&nflt=ht_id%3D204&srpvid=273d4af163be01eb&srepoch=1706784087&all_sr_blocks=1133014901_385969623_2_0_0&highlighted_blocks=1133014901_385969623_2_0_0&matching_block_id=1133014901_385969623_2_0_0&sr_pri_blocks=1133014901_385969623_2_0_0__113400&from=searchresults#hotelTmpl
327 out of 353
https://www.booking.com/hotel/es/la-quinta-de-los-cedros.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Aqvp7a0GwAIB0gIkMTIwY2VhY2QtNzJlMi00Yjg2LWJmYmEtMGZiNTI3YTc4NmVi2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-17&checkout=2024-06-23&dest_id=-390625&dest_type=city&

https://www.booking.com/hotel/es/villamadrid.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Aqvp7a0GwAIB0gIkMTIwY2VhY2QtNzJlMi00Yjg2LWJmYmEtMGZiNTI3YTc4NmVi2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-17&checkout=2024-06-23&dest_id=-390625&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=13&hapos=338&sr_order=popularity&nflt=ht_id%3D204&srpvid=273d4af163be01eb&srepoch=1706784087&all_sr_blocks=9426101_88195701_0_2_0&highlighted_blocks=9426101_88195701_0_2_0&matching_block_id=9426101_88195701_0_2_0&sr_pri_blocks=9426101_88195701_0_2_0__85600&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
339 out of 353
https://www.booking.com/hotel/es/chic-basic-atocha.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Aqvp7a0GwAIB0gIkMTIwY2VhY2QtNzJlMi00Yjg2LWJmYmEtMGZiNTI3YTc4NmVi2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-17&checkout=2024-06-23&dest_id=-390625&dest_type=city&

https://www.booking.com/hotel/es/barajas-plaza-sl.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Aqvp7a0GwAIB0gIkMTIwY2VhY2QtNzJlMi00Yjg2LWJmYmEtMGZiNTI3YTc4NmVi2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-17&checkout=2024-06-23&dest_id=-390625&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=25&hapos=350&sr_order=popularity&nflt=ht_id%3D204&srpvid=273d4af163be01eb&srepoch=1706784087&all_sr_blocks=1213802_95135372_2_0_0&highlighted_blocks=1213802_95135372_2_0_0&matching_block_id=1213802_95135372_2_0_0&sr_pri_blocks=1213802_95135372_2_0_0__96000&from=searchresults#hotelTmpl
351 out of 353
https://www.booking.com/hotel/es/osuna.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Aqvp7a0GwAIB0gIkMTIwY2VhY2QtNzJlMi00Yjg2LWJmYmEtMGZiNTI3YTc4NmVi2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-17&checkout=2024-06-23&dest_id=-390625&dest_type=city&group_adults=2&req_adults=2&no_rooms=1

In [49]:
start_date = '2024-06-03'
end_date = '2024-06-09'
place = "Madrid, Spain"
file_path = start_date +'_'+end_date+'_'+place+'.csv'
data_c2_w2 = search_hotels(browser=browser, start_date=start_date,end_date=end_date,place=place)
data_c2_w2 = get_hotel_details(data_c2_w2)

Message: Unable to locate element: //button[@aria-label="Dismiss sign-in info."]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:509:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Message: Unable to locate element: //button[@aria-label="Dismiss sign-in info."]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:509:5
dom.find

Message: Unable to locate element: /html/body/div[3]/div[2]/div/form/div[1]/div[4]/button; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:509:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Message: Unable to locate element: /html/body/div[3]/div[2]/div/form/div[1]/div[4]/button; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.

No original price
No sustainability information
No comfort rating
No original price
No original price
No comfort rating
No original price
No comfort rating
No original price
[<selenium.webdriver.remote.webelement.WebElement (session="e94a43f7-2991-4df0-89ed-3dfd66447dfd", element="dbc523bd-d39f-4d2e-8e91-334e6c5adbaa")>]
No comfort rating
No original price
No comfort rating
No comfort rating
No original price
No comfort rating
No original price
No original price
No sustainability information
No original price
No comfort rating
No original price
No original price
No original price
No comfort rating
No original price
No original price
No comfort rating
No original price
No sustainability information
No comfort rating
No original price
No original price
No comfort rating
No original price
No comfort rating
No original price
No sustainability information
No original price
No comfort rating
No original price
No sustainability information
No comfort rating
No original price
No sustainability

No original price
No comfort rating
No original price
No comfort rating
No original price
No comfort rating
No original price
No comfort rating
No original price
No original price
No original price
No comfort rating
No original price
No original price
No original price
No original price
No sustainability information
No comfort rating
No original price
No original price
[<selenium.webdriver.remote.webelement.WebElement (session="e94a43f7-2991-4df0-89ed-3dfd66447dfd", element="dbc523bd-d39f-4d2e-8e91-334e6c5adbaa")>]
No comfort rating
No original price
No original price
No original price
No original price
No sustainability information
No comfort rating
No original price
No sustainability information
No original price
No original price
No original price
No comfort rating
No original price
No original price
No original price
No comfort rating
No original price
No original price
No original price
No comfort rating
No original price
No original price
No original price
No original price
No co

https://www.booking.com/hotel/es/axel-madrid-madrid12345678.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Arn07a0GwAIB0gIkYzI0OTVjNGEtMGRiMy00YjczLWIxZmUtMGJhM2E1NGJhNjUx2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&dest_id=-390625&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=3&hapos=3&sr_order=popularity&nflt=ht_id%3D204&srpvid=ca9c4db85b0d002a&srepoch=1706785414&all_sr_blocks=243898501_249063127_0_2_0&highlighted_blocks=243898501_249063127_0_2_0&matching_block_id=243898501_249063127_0_2_0&sr_pri_blocks=243898501_249063127_0_2_0__112541&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
3 out of 376
https://www.booking.com/hotel/es/velada-madrid.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Arn07a0GwAIB0gIkYzI0OTVjNGEtMGRiMy00YjczLWIxZmUtMGJhM2E1NGJhNjUx2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&dest_id=-390

...

https://www.booking.com/hotel/es/soho-boutique-congreso.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Arn07a0GwAIB0gIkYzI0OTVjNGEtMGRiMy00YjczLWIxZmUtMGJhM2E1NGJhNjUx2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&dest_id=-390625&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=326&sr_order=popularity&nflt=ht_id%3D204&srpvid=ca9c4db85b0d002a&srepoch=1706785505&all_sr_blocks=393404501_122676298_0_2_0&highlighted_blocks=393404501_122676298_0_2_0&matching_block_id=393404501_122676298_0_2_0&sr_pri_blocks=393404501_122676298_0_2_0__108900&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
327 out of 376
https://www.booking.com/hotel/es/barcelo-castellana-norte.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Arn07a0GwAIB0gIkYzI0OTVjNGEtMGRiMy00YjczLWIxZmUtMGJhM2E1NGJhNjUx2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&d

https://www.booking.com/hotel/es/casualmadriddelteatro.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Arn07a0GwAIB0gIkYzI0OTVjNGEtMGRiMy00YjczLWIxZmUtMGJhM2E1NGJhNjUx2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&dest_id=-390625&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=13&hapos=338&sr_order=popularity&nflt=ht_id%3D204&srpvid=ca9c4db85b0d002a&srepoch=1706785505&all_sr_blocks=0_0_2_1_0&highlighted_blocks=0_0_2_1_0&matching_block_id=0_0_2_1_0&sr_pri_blocks=0_0_2_1_0__107280&from=searchresults#hotelTmpl
339 out of 376
https://www.booking.com/hotel/es/mexico-madrid.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Arn07a0GwAIB0gIkYzI0OTVjNGEtMGRiMy00YjczLWIxZmUtMGJhM2E1NGJhNjUx2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&dest_id=-390625&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=

https://www.booking.com/hotel/es/barajas-plaza-sl.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Arn07a0GwAIB0gIkYzI0OTVjNGEtMGRiMy00YjczLWIxZmUtMGJhM2E1NGJhNjUx2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&dest_id=-390625&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=25&hapos=350&sr_order=popularity&nflt=ht_id%3D204&srpvid=ca9c4db85b0d002a&srepoch=1706785505&all_sr_blocks=1213802_95135372_2_0_0&highlighted_blocks=1213802_95135372_2_0_0&matching_block_id=1213802_95135372_2_0_0&sr_pri_blocks=1213802_95135372_2_0_0__84000&from=searchresults#hotelTmpl
351 out of 376
https://www.booking.com/hotel/es/soho-boutique-congreso.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Arn07a0GwAIB0gIkYzI0OTVjNGEtMGRiMy00YjczLWIxZmUtMGJhM2E1NGJhNjUx2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&dest_id=-390625&dest_type=city&group_adults=2&req_ad

https://www.booking.com/hotel/es/gran-via-capital-madrid1.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Arn07a0GwAIB0gIkYzI0OTVjNGEtMGRiMy00YjczLWIxZmUtMGJhM2E1NGJhNjUx2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&dest_id=-390625&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=12&hapos=337&sr_order=popularity&nflt=ht_id%3D204&srpvid=ca9c4db85b0d002a&srepoch=1706785505&all_sr_blocks=811577801_0_2_0_0&highlighted_blocks=811577801_0_2_0_0&matching_block_id=811577801_0_2_0_0&sr_pri_blocks=811577801_0_2_0_0__161879&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
363 out of 376
https://www.booking.com/hotel/es/casualmadriddelteatro.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Arn07a0GwAIB0gIkYzI0OTVjNGEtMGRiMy00YjczLWIxZmUtMGJhM2E1NGJhNjUx2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&dest_id=-390625&dest_type=city&gr

https://www.booking.com/hotel/es/amor-de-dios-17.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Arn07a0GwAIB0gIkYzI0OTVjNGEtMGRiMy00YjczLWIxZmUtMGJhM2E1NGJhNjUx2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&dest_id=-390625&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=24&hapos=349&sr_order=popularity&nflt=ht_id%3D204&srpvid=ca9c4db85b0d002a&srepoch=1706785505&all_sr_blocks=294063001_109580026_2_0_0&highlighted_blocks=294063001_109580026_2_0_0&matching_block_id=294063001_109580026_2_0_0&sr_pri_blocks=294063001_109580026_2_0_0__162864&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
375 out of 376
https://www.booking.com/hotel/es/barajas-plaza-sl.html?label=gen173nr-1FCAEoggI46AdIM1gEaEaIAQGYATG4ARnIAQ_YAQHoAQH4AQKIAgGoAgO4Arn07a0GwAIB0gIkYzI0OTVjNGEtMGRiMy00YjczLWIxZmUtMGJhM2E1NGJhNjUx2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-06-03&checkout=2024-06-09&dest_id=-390625

In [50]:
def clean_df(df):
  # Remove strings and symbols from the data
  df['central_distance'] = pd.to_numeric(df['central_distance'].str.replace(r'[a-zA-Z]','', regex=True))
  df['comfort_rating'] = pd.to_numeric(df['comfort_rating'].str.replace(r'[a-zA-Z]','', regex=True))
  df['sustainability'] = pd.to_numeric(df['sustainability'].str.replace(r'[a-zA-Z+]','', regex=True))
  df['review_count'] = pd.to_numeric(df['review_count'].str.replace(r'[a-zA-Z+,]','', regex=True))
  df['price'] = pd.to_numeric(df['price'].str.replace(r'[a-zA-Z€,]','', regex=True))
  df['original_price'] = pd.to_numeric(df['original_price'].str.replace(r'[a-zA-Z€,]','', regex=True))

  # Change date type data
  df['start_date'] = pd.to_datetime(df['start_date'])
  df['start_date'] = df['start_date'].dt.strftime('%Y/%m/%d')
  df['end_date'] = pd.to_datetime(df['end_date'])
  df['end_date'] = df['end_date'].dt.strftime('%Y/%m/%d')
    
  return df

# Function to remove the discount info
def remove_info(text):
    # Define the regex pattern to match the text to be removed
    pattern1 = re.compile(r"You're eligible for a Genius discount.*?all you have to do is sign in\.")
    pattern2 = re.compile(r'Couples in particular like the location –.*?Show me more\n\n\n\n', re.DOTALL)

    # Replace the matched portion with an empty string
    cleaned_text = re.sub(pattern1, '', text)
    cleaned_text = re.sub(pattern2, '', cleaned_text)
    cleaned_text = re.sub(r'\n+', '\n', cleaned_text)
    cleaned_text = cleaned_text.strip('\n')
    
    return cleaned_text

In [51]:
data_c1_w1 = clean_df(data_c1_w1)
data_c1_w1['description'] = data_c1_w1['description'].apply(remove_info)
data_c1_w2 = clean_df(data_c1_w2)
data_c1_w2['description'] = data_c1_w2['description'].apply(remove_info)
data_c2_w1 = clean_df(data_c2_w1)
data_c2_w1['description'] = data_c2_w1['description'].apply(remove_info)
data_c2_w2 = clean_df(data_c2_w2)
data_c2_w2['description'] = data_c2_w2['description'].apply(remove_info)

data_c1_w1.to_csv('2024-06-17_2024-06-23_BCN_cleaned.csv')
data_c1_w2.to_csv('2024-06-03_2024-06-09_BCN_cleaned.csv')
data_c2_w1.to_csv('2024-06-17_2024-06-23_MAD_cleaned.csv')
data_c2_w2.to_csv('2024-06-03_2024-06-09_MAD_cleaned.csv')



In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')


In [91]:
data_copy = pd.read_csv('2024-03-03_2024-03-10_Los Ángeles, California, Estados Unidos.csv')
data_copy = clean_df(data_copy)
data_copy['description'] = data_copy['description'].apply(remove_info)

sentences = data_copy['description'].tolist()

sentences

['Hollywood Homes minutes to everything SPACIOUS AND FREE PARKING is a sustainable condo hotel in Los Angeles, 2 miles from Capitol Records Building. This property encourages guests to embrace nature with its eco-friendly arrangements. This condo hotel offers free private parking, a 24-hour front desk, and free Wifi. The condo hotel has private entrance.\nAt the condo hotel, units are equipped with air conditioning, a seating area, a flat-screen TV with streaming services, a kitchen, a dining area, and a private bathroom with a hair dryer, a bath, and free toiletries. A dishwasher, an oven, and microwave are also offered, as well as a coffee machine and a kettle. At the condo hotel, the units come with bed linen and towels.\nThe condo hotel has a picnic area where you can spend the day outdoors.\nDolby Theater is 2.9 miles from the condo hotel, while Hollywood Bowl is 3.1 miles from the property. The nearest airport is Hollywood Burbank Airport, 8.7 miles from Hollywood Homes minutes t

In [ ]:
from nltk.corpus import stopwords

def strip(word):
    mod_string = re.sub(r'\W+', '', word)
    return mod_string

def abbr_or_lower(word):
    if re.match('([A-Z]+[a-z]*){2,}', word):
        return word
    else:
        return word.lower()
    
corpus_stop=[]
for index, row in enumerate(sentences):
    words = row.split()
    lowered = [abbr_or_lower(strip(w)) for w in words if abbr_or_lower(strip(w)) not in set(stopwords.words('english'))]
    lowered_sentence = ' '.join(lowered)
    print(lowered_sentence)
    corpus_stop.append(lowered)
    
    # Replace the original sentence with the lowercased version in the 'sentences' list
    sentences[index] = lowered_sentence

sentences

In [94]:
from nltk.stem.snowball import SnowballStemmer
englishStemmer=SnowballStemmer("english")

corpus_stem=[]
for sentence in sentences:
    words = sentence.split()
    lowered = [englishStemmer.stem(strip(w)) for w in words]
    print(lowered)
    corpus_stem.append(lowered)

['hollywood', 'home', 'minut', 'everyth', 'spacious', 'and', 'free', 'park', 'sustain', 'condo', 'hotel', 'los', 'angel', '2', 'mile', 'capitol', 'record', 'build', 'properti', 'encourag', 'guest', 'embrac', 'natur', 'ecofriend', 'arrang', 'condo', 'hotel', 'offer', 'free', 'privat', 'park', '24hour', 'front', 'desk', 'free', 'wifi', 'condo', 'hotel', 'privat', 'entranc', 'condo', 'hotel', 'unit', 'equip', 'air', 'condit', 'seat', 'area', 'flatscreen', 'tv', 'stream', 'servic', 'kitchen', 'dine', 'area', 'privat', 'bathroom', 'hair', 'dryer', 'bath', 'free', 'toiletri', 'dishwash', 'oven', 'microwav', 'also', 'offer', 'well', 'coffe', 'machin', 'kettl', 'condo', 'hotel', 'unit', 'come', 'bed', 'linen', 'towel', 'condo', 'hotel', 'picnic', 'area', 'spend', 'day', 'outdoor', 'dolbi', 'theater', '29', 'mile', 'condo', 'hotel', 'hollywood', 'bowl', '31', 'mile', 'properti', 'nearest', 'airport', 'hollywood', 'burbank', 'airport', '87', 'mile', 'hollywood', 'home', 'minut', 'everyth', 'spac

In [95]:
vocab, index = {}, 1  # start indexing from 1
for doc in corpus_stem:
    for token in doc:
        if token not in vocab:
            vocab[token] = index
            index += 1
        
vocab_stem=vocab
vocab_stem_size = len(vocab)
print(vocab_stem)
print(" ")
print("Total size of stemmed vocabulary:", vocab_stem_size)

{'hollywood': 1, 'home': 2, 'minut': 3, 'everyth': 4, 'spacious': 5, 'and': 6, 'free': 7, 'park': 8, 'sustain': 9, 'condo': 10, 'hotel': 11, 'los': 12, 'angel': 13, '2': 14, 'mile': 15, 'capitol': 16, 'record': 17, 'build': 18, 'properti': 19, 'encourag': 20, 'guest': 21, 'embrac': 22, 'natur': 23, 'ecofriend': 24, 'arrang': 25, 'offer': 26, 'privat': 27, '24hour': 28, 'front': 29, 'desk': 30, 'wifi': 31, 'entranc': 32, 'unit': 33, 'equip': 34, 'air': 35, 'condit': 36, 'seat': 37, 'area': 38, 'flatscreen': 39, 'tv': 40, 'stream': 41, 'servic': 42, 'kitchen': 43, 'dine': 44, 'bathroom': 45, 'hair': 46, 'dryer': 47, 'bath': 48, 'toiletri': 49, 'dishwash': 50, 'oven': 51, 'microwav': 52, 'also': 53, 'well': 54, 'coffe': 55, 'machin': 56, 'kettl': 57, 'come': 58, 'bed': 59, 'linen': 60, 'towel': 61, 'picnic': 62, 'spend': 63, 'day': 64, 'outdoor': 65, 'dolbi': 66, 'theater': 67, '29': 68, 'bowl': 69, '31': 70, 'nearest': 71, 'airport': 72, 'burbank': 73, '87': 74, 'overlook': 75, 'pacif': 

In [96]:
def vectorize(tokens, vocab):
    vector=[]
    for w in vocab:
        vector.append(tokens.count(w))
    return vector

vectors=[]
for doc in corpus_stem:
    vectors.append(vectorize(doc, vocab_stem))

In [97]:
#making a pandas dataframe
df = pd.DataFrame(vectors)
df

,0,1,2,3,4,5,6,7,8,9,...,2161,2162,2163,2164,2165,2166,2167,2168,2169,2170
0,4,2,2,2,2,2,5,3,1,7,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,3,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
293,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
294,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
295,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [106]:
from sklearn.feature_extraction.text import CountVectorizer

data_copy = pd.read_csv('2024-03-03_2024-03-10_Los Ángeles, California, Estados Unidos.csv')
data_copy = clean_df(data_copy)
data_copy['description'] = data_copy['description'].apply(remove_info)

sentences = data_copy['description'].tolist()

cv = CountVectorizer(ngram_range = (1,2), min_df=0.05, max_df=0.5)
cv.fit(sentences)
vectorized_text=cv.transform(sentences)
vectorized_text=vectorized_text.todense()
print("document term matrix has size", vectorized_text.shape)
print(cv.get_feature_names_out())

document term matrix has size (297, 635)
['10' '10 minute' '10 minutes' '11' '11 mi' '11 miles' '12' '15'
 '15 minutes' '20' '24' '24 hour' 'access' 'access to' 'accommodation'
 'accommodation while' 'accommodations' 'accommodations and'
 'accommodations with' 'air' 'air conditioned' 'air conditioning'
 'airport 11' 'airport miles' 'all' 'all guest' 'all rooms' 'along'
 'along with' 'also available' 'also provided' 'amenities' 'american'
 'an on' 'an outdoor' 'an oven' 'and 24' 'and an' 'and bar' 'and coffee'
 'and dinner' 'and free' 'and guests' 'and hairdryer' 'and is' 'and mi'
 'and microwave' 'and miles' 'and private' 'and santa' 'and terrace'
 'and the' 'and towels' 'and work' 'angeles convention' 'angeles county'
 'angeles hotel' 'angeles international' 'angeles is' 'angeles miles'
 'angeles the' 'are also' 'are available' 'are equipped' 'are offered'
 'are provided' 'are within' 'area' 'area and' 'around' 'around miles'
 'art' 'art lacma' 'as' 'as well' 'at this' 'attractions' '

In [107]:
for document in vectorized_text:
    print(document)

[[0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1
  0 0 0 0 0 4 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0 1 0 0 0 3 1 0 0 0
  0 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0
  1 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
  0 1 1 0 0 0 0 0 0 1 7 7 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1
  1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 2 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 1 0 0 2 0 0 0 0 0 0 0 0 0 4 1 1 0 0 0 0 0 0 0 0 0 0
  2 0 0 1 0 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 1 0 0 0 0 0 2 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0
  0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 4 0 0 0 0 0 0 2 0 0 0
  0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 1 0 0 0 0 3 1 0 0 0 0 0 0 0 0 0 3 1 1 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [109]:
from sklearn.feature_extraction.text import TfidfVectorizer

cv = TfidfVectorizer(ngram_range = (1,2), min_df=0.05, max_df=0.5, norm=None)
cv.fit(sentences)
vectorized_text=cv.transform(sentences)
vectorized_text=vectorized_text.todense()
print("document term matrix has size", vectorized_text.shape)
print(cv.get_feature_names_out())

document term matrix has size (297, 635)
['10' '10 minute' '10 minutes' '11' '11 mi' '11 miles' '12' '15'
 '15 minutes' '20' '24' '24 hour' 'access' 'access to' 'accommodation'
 'accommodation while' 'accommodations' 'accommodations and'
 'accommodations with' 'air' 'air conditioned' 'air conditioning'
 'airport 11' 'airport miles' 'all' 'all guest' 'all rooms' 'along'
 'along with' 'also available' 'also provided' 'amenities' 'american'
 'an on' 'an outdoor' 'an oven' 'and 24' 'and an' 'and bar' 'and coffee'
 'and dinner' 'and free' 'and guests' 'and hairdryer' 'and is' 'and mi'
 'and microwave' 'and miles' 'and private' 'and santa' 'and terrace'
 'and the' 'and towels' 'and work' 'angeles convention' 'angeles county'
 'angeles hotel' 'angeles international' 'angeles is' 'angeles miles'
 'angeles the' 'are also' 'are available' 'are equipped' 'are offered'
 'are provided' 'are within' 'area' 'area and' 'around' 'around miles'
 'art' 'art lacma' 'as' 'as well' 'at this' 'attractions' '

In [110]:
vectorized_text=vectorized_text.round(decimals=3, out=None)
for document in vectorized_text:
    print(document)

[ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  1.853  1.97   0.     0.     0.     0.     0.     0.     0.     1.792
  0.     3.114  0.     2.554  0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     3.864  0.     0.     0.     0.
  0.     9.91   0.     0.     0.     0.     3.864  0.     3.086  0.
  0.     0.     2.936  0.     0.     0.     0.     0.     0.     2.89
  0.     2.538  0.     2.913  0.     0.     0.     6.694  3.753  0.
  0.     0.     0.     5.14   2.847  0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     3.365
  2.22   3.201  0.     0.     0.     0.     0.     0.     2.554  2.746
  0.     0.     0.     0.     0.     0.     0.     0.     2.746  3.171
  0.     0.     2.603  3.231  3.231  0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     2.69   2.69   0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     2.062  2.826  0.  

In [145]:
data_c1_w1 = pd.read_csv('2024-06-03_2024-06-09_BCN_cleaned.csv')
data_c1_w2 = pd.read_csv('2024-06-17_2024-06-23_BCN_cleaned.csv')
data_c2_w1 = pd.read_csv('2024-06-03_2024-06-09_MAD_cleaned.csv')
data_c2_w2 = pd.read_csv('2024-06-17_2024-06-23_MAD_cleaned.csv')
#data_c1_w1['review_count'] = pd.to_numeric(data_c1_w1['review_count'].str.replace(r'[a-zA-Z+,]','', regex=True))
#data_c1_w2['review_count'] = pd.to_numeric(data_c1_w2['review_count'].str.replace(r'[a-zA-Z+,]','', regex=True))
#data_c2_w1['review_count'] = pd.to_numeric(data_c2_w1['review_count'].str.replace(r'[a-zA-Z+,]','', regex=True))
#data_c2_w2['review_count'] = pd.to_numeric(data_c2_w2['review_count'].str.replace(r'[a-zA-Z+,]','', regex=True))


In [146]:
data_c1_w1.head()

,Unnamed: 0,photo_url,link,name,central_distance,sustainability,rating,comfort_rating,review_count,room_description,room_specifics,price,original_price,neighborhood_city,start_date,end_date,name.1,description
0,0,https://cf.bstatic.com/xdata/images/hotel/squa...,https://www.booking.com/hotel/es/sixtytwo-barc...,Sixtytwo Hotel,0.8,1.0,9.2,9.8,1833.0,Standard Double Room,Beds: 1 double or 2 twins,1857,3051.0,"Eixample, Barcelona",2024/06/03,2024/06/09,Sixtytwo Hotel,Located next to Barcelona’s Passeig de Gracia ...
1,1,https://cf.bstatic.com/xdata/images/hotel/squa...,https://www.booking.com/hotel/es/melia-barcelo...,Melia Barcelona Sky 4* Sup,3.3,1.0,8.2,NaN,1291.0,The Level Grand Premium Room City View,Beds: 1 double or 2 twins,196342224,NaN,"Sant Martí, Barcelona",2024/06/03,2024/06/09,Melia Barcelona Sky 4* Sup,"One of Barcelona’s tallest buildings, this des..."
2,2,https://cf.bstatic.com/xdata/images/hotel/squa...,https://www.booking.com/hotel/es/sansi-barcelo...,Hotel Sansi Barcelona,450.0,NaN,8.0,NaN,780.0,Basic Double,1 full bed,1034,NaN,"Eixample, Barcelona",2024/06/03,2024/06/09,Hotel Sansi Barcelona,In a prime location in the Eixample district o...
3,3,https://cf.bstatic.com/xdata/images/hotel/squa...,https://www.booking.com/hotel/es/hotelalimara....,Hotel Alimara,5.5,1.0,8.3,NaN,3894.0,Double or Twin Room,2 twin beds,659,NaN,"Horta-Guinardó, Barcelona",2024/06/03,2024/06/09,Hotel Alimara,Hotel Alimara is a modern hotel located in Bar...
4,4,https://cf.bstatic.com/xdata/images/hotel/squa...,https://www.booking.com/hotel/es/cataloniatena...,Catalonia Atenas,3.0,3.0,8.2,NaN,7692.0,Double or Twin Room,Beds: 1 double or 2 twins,1029,NaN,"Sant Martí, Barcelona",2024/06/03,2024/06/09,Catalonia Atenas,"Set 3 metro stops from central Barcelona, the ..."


In [147]:
data_c1_w1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 437 entries, 0 to 436
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         437 non-null    int64  
 1   photo_url          437 non-null    object 
 2   link               437 non-null    object 
 3   name               437 non-null    object 
 4   central_distance   437 non-null    float64
 5   sustainability     342 non-null    float64
 6   rating             434 non-null    float64
 7   comfort_rating     197 non-null    float64
 8   review_count       434 non-null    float64
 9   room_description   437 non-null    object 
 10  room_specifics     435 non-null    object 
 11  price              437 non-null    int64  
 12  original_price     133 non-null    float64
 13  neighborhood_city  437 non-null    object 
 14  start_date         437 non-null    object 
 15  end_date           437 non-null    object 
 16  name.1             437 non

In [148]:
data_c1_w1.columns

Index(['Unnamed: 0', 'photo_url', 'link', 'name', 'central_distance',
       'sustainability', 'rating', 'comfort_rating', 'review_count',
       'room_description', 'room_specifics', 'price', 'original_price',
       'neighborhood_city', 'start_date', 'end_date', 'name.1', 'description'],
      dtype='object')

In [149]:
data_c1_w1.drop(columns=['Unnamed: 0', 'photo_url', 'link', 'name', 'original_price', 'name.1'], inplace=True)

In [150]:
data_c1_w1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 437 entries, 0 to 436
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   central_distance   437 non-null    float64
 1   sustainability     342 non-null    float64
 2   rating             434 non-null    float64
 3   comfort_rating     197 non-null    float64
 4   review_count       434 non-null    float64
 5   room_description   437 non-null    object 
 6   room_specifics     435 non-null    object 
 7   price              437 non-null    int64  
 8   neighborhood_city  437 non-null    object 
 9   start_date         437 non-null    object 
 10  end_date           437 non-null    object 
 11  description        437 non-null    object 
dtypes: float64(5), int64(1), object(6)
memory usage: 41.1+ KB


In [151]:
num_feat = ['central_distance', 'sustainability', 'rating', 'comfort_rating', 'review_count']
cat_feat = ['room_description', 'room_specifics', 'neighborhood_city']

for c in cat_feat:
    print(f'{c}:')
    print(data_c1_w1[c].unique())

room_description:
['Standard Double Room' 'The Level Grand Premium Room City View'
 'Basic Double' 'Double or Twin Room' 'Superior Double Room'
 'Standard Double or Twin Room' 'Double Room with Spa Access'
 'Preferred King Room with City View' 'Double Room' 'Twin Room'
 'Standard Twin Room' 'Room Assigned on Arrival' 'Twin Room with Balcony'
 'Small Double Room' 'Double or Twin Room (1-2 Adults)'
 'Basic Double or Twin Room' 'Deluxe Studio' 'Comfort Double or Twin Room'
 'Superior Double or Twin Room with Terrace'
 'Interior Comfort Double or Twin Room' 'Superior Double or Twin Room'
 'Basic Double Room' 'One-Bedroom Apartment'
 'Standard Duplex Room (1-2 Adults)'
 'Comfort Double or Twin Room (1-2 Adults)'
 'Double or Twin Room with Private Bathroom'
 'Standard Double or Twin Room with Balcony'
 'One-Bedroom Apartment (2 Adults)' 'Apartment (2 Adults)'
 'Comfort Quadruple Room' 'Superior Double or Twin Room (1-2 Adults)'
 'Standard Double Exterior Room' 'Budget Twin Room' 'Comfort Dou

In [152]:
# Function to extract the first number from a text
def extract_first_number(text):
    # Regular expression pattern
    pattern = re.compile(r'\b\d+\b')

    try:
        match = pattern.search(text)
        return int(match.group()) if match else None
    except:
        return None

data_c1_w1['num_beds'] = data_c1_w1['room_specifics'].apply(lambda x: extract_first_number(x))
data_c1_w1.drop(columns=['room_description', 'neighborhood_city', 'room_specifics'], inplace=True)
data_c1_w1['city'] = 'Barcelona'

In [153]:
data_c1_w1.sort_values('price', inplace=True)
for c in num_feat+['num_beds']:
    data_c1_w1[c].ffill(inplace=True)
    data_c1_w1[c].bfill(inplace=True)

data_c1_w1['num_beds'] = data_c1_w1['num_beds'].astype(int)

In [154]:
data_c1_w1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 437 entries, 59 to 1
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   central_distance  437 non-null    float64
 1   sustainability    437 non-null    float64
 2   rating            437 non-null    float64
 3   comfort_rating    437 non-null    float64
 4   review_count      437 non-null    float64
 5   price             437 non-null    int64  
 6   start_date        437 non-null    object 
 7   end_date          437 non-null    object 
 8   description       437 non-null    object 
 9   num_beds          437 non-null    int32  
 10  city              437 non-null    object 
dtypes: float64(5), int32(1), int64(1), object(4)
memory usage: 39.3+ KB


In [155]:
data_c1_w2['num_beds'] = data_c1_w2['room_specifics'].apply(lambda x: extract_first_number(x))
data_c1_w2.drop(columns=['Unnamed: 0', 'photo_url', 'link', 'name', 'original_price', 'name.1', 'room_description', 'neighborhood_city', 'room_specifics'], inplace=True)
data_c1_w2['city'] = 'Barcelona'
data_c1_w2.sort_values('price', inplace=True)
for c in num_feat+['num_beds']:
    data_c1_w2[c].ffill(inplace=True)
    data_c1_w2[c].bfill(inplace=True)

data_c1_w2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 378 entries, 305 to 285
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   central_distance  378 non-null    float64
 1   sustainability    378 non-null    float64
 2   rating            378 non-null    float64
 3   comfort_rating    378 non-null    float64
 4   review_count      378 non-null    float64
 5   price             378 non-null    int64  
 6   start_date        378 non-null    object 
 7   end_date          378 non-null    object 
 8   description       378 non-null    object 
 9   num_beds          378 non-null    float64
 10  city              378 non-null    object 
dtypes: float64(6), int64(1), object(4)
memory usage: 35.4+ KB


In [156]:
data_c2_w1['num_beds'] = data_c2_w1['room_specifics'].apply(lambda x: extract_first_number(x))
data_c2_w1.drop(columns=['Unnamed: 0', 'photo_url', 'link', 'name', 'original_price', 'name.1', 'room_description', 'neighborhood_city', 'room_specifics'], inplace=True)
data_c2_w1['city'] = 'Madrid'
data_c2_w1.sort_values('price', inplace=True)
for c in num_feat+['num_beds']:
    data_c2_w1[c].ffill(inplace=True)
    data_c2_w1[c].bfill(inplace=True)

data_c2_w1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 376 entries, 295 to 220
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   central_distance  376 non-null    float64
 1   sustainability    376 non-null    float64
 2   rating            376 non-null    float64
 3   review_count      376 non-null    float64
 4   price             376 non-null    int64  
 5   comfort_rating    376 non-null    float64
 6   start_date        376 non-null    object 
 7   end_date          376 non-null    object 
 8   description       376 non-null    object 
 9   num_beds          376 non-null    float64
 10  city              376 non-null    object 
dtypes: float64(6), int64(1), object(4)
memory usage: 35.2+ KB


In [157]:
data_c2_w2['num_beds'] = data_c2_w2['room_specifics'].apply(lambda x: extract_first_number(x))
data_c2_w2.drop(columns=['Unnamed: 0', 'photo_url', 'link', 'name', 'original_price', 'name.1', 'room_description', 'neighborhood_city', 'room_specifics'], inplace=True)
data_c2_w2['city'] = 'Madrid'
data_c2_w2.sort_values('price', inplace=True)
for c in num_feat+['num_beds']:
    data_c2_w2[c].ffill(inplace=True)
    data_c2_w2[c].bfill(inplace=True)

data_c2_w2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 353 entries, 322 to 248
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   central_distance  353 non-null    float64
 1   sustainability    353 non-null    float64
 2   rating            353 non-null    float64
 3   review_count      353 non-null    float64
 4   price             353 non-null    int64  
 5   comfort_rating    353 non-null    float64
 6   start_date        353 non-null    object 
 7   end_date          353 non-null    object 
 8   description       353 non-null    object 
 9   num_beds          353 non-null    float64
 10  city              353 non-null    object 
dtypes: float64(6), int64(1), object(4)
memory usage: 33.1+ KB


In [158]:
# First equation
data_c1_w1['event'] = 0
data_c1_w2['event'] = 1

data_eq1 = pd.concat([data_c1_w1, data_c1_w2]).drop(columns=['city', 'start_date', 'end_date', 'description'])
data_eq1 = data_eq1[data_eq1['price'] <= 1000000] #there is some real big outlier
data_eq1.head()

,central_distance,sustainability,rating,comfort_rating,review_count,price,num_beds,event
59,5.5,3.0,7.2,9.3,1047.0,581,2.0,0
3,5.5,1.0,8.3,9.3,3894.0,659,2.0,0
68,5.7,3.0,8.3,9.3,2879.0,713,1.0,0
186,2.6,3.0,8.0,9.3,1562.0,774,1.0,0
238,2.6,3.0,8.0,9.3,840.0,774,3.0,0


In [159]:
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

# Create a LinearRegression model
model = LinearRegression()
features = ['central_distance', 'sustainability', 'rating', 'comfort_rating',
    'review_count', 'num_beds', 'event']

# scale continuous variables
scaler = preprocessing.StandardScaler()
scaler.fit(data_eq1[features])## Fitted to the training set

X_data = scaler.transform(data_eq1[features])
X_data=pd.DataFrame(X_data, columns=features)

# Fit the model to the data_eq1
model.fit(X_data, data_eq1['price'])

# Extract the coefficients
beta_0 = round(model.intercept_, 2)
beta_1 = round(model.coef_[0], 2)
beta_2 = round(model.coef_[1], 2)
beta_3 = round(model.coef_[2], 2)
beta_4 = round(model.coef_[3], 2)
beta_5 = round(model.coef_[4], 2)
beta_6 = round(model.coef_[5], 2)
beta_7 = round(model.coef_[6], 2)

print("Beta_0:", beta_0)
print("Beta for  central distance: ", beta_1)
print("Beta for sustainability: ", beta_2)
print("Beta for rating: ", beta_3)
print("Beta for comfort rating: ", beta_4)
print("Beta for review count: ", beta_5)
print("Betas for number of beds: ", beta_6)
print("Betas for dummy event: ", beta_7)

Beta_0: 1735.23
Beta for  central distance:  44.3
Beta for sustainability:  106.92
Beta for rating:  247.08
Beta for comfort rating:  157.06
Beta for review count:  -122.01
Betas for number of beds:  91.88
Betas for dummy event:  147.03


In [160]:
import numpy as np

# Second equation
data_c1_w1.drop(columns=['event'], inplace=True)
data_c1_w2.drop(columns=['event'], inplace=True)

data_eq2 = pd.concat([data_c1_w1, data_c2_w1]).drop(columns=['start_date', 'end_date', 'description'])
data_eq2['dummy_city'] = np.where(data_eq2['city'] == 'Barcelona',1 ,0)
data_eq2.drop(columns=['city'], inplace=True)
data_eq2 = data_eq2[data_eq2['price'] <= 1000000] #there is some real big outlier
data_eq2.head()

,central_distance,sustainability,rating,comfort_rating,review_count,price,num_beds,dummy_city
59,5.5,3.0,7.2,9.3,1047.0,581,2.0,1
3,5.5,1.0,8.3,9.3,3894.0,659,2.0,1
68,5.7,3.0,8.3,9.3,2879.0,713,1.0,1
186,2.6,3.0,8.0,9.3,1562.0,774,1.0,1
238,2.6,3.0,8.0,9.3,840.0,774,3.0,1


In [161]:
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

# Create a LinearRegression model
model = LinearRegression()
features = ['central_distance', 'sustainability', 'rating', 'comfort_rating',
    'review_count', 'num_beds', 'dummy_city']

# scale continuous variables
scaler = preprocessing.StandardScaler()
scaler.fit(data_eq2[features])## Fitted to the training set

X_data = scaler.transform(data_eq2[features])
X_data=pd.DataFrame(X_data, columns=features)

# Fit the model to the data_eq1
model.fit(X_data, data_eq2['price'])

# Extract the coefficients
beta_0 = round(model.intercept_, 2)
beta_1 = round(model.coef_[0], 2)
beta_2 = round(model.coef_[1], 2)
beta_3 = round(model.coef_[2], 2)
beta_4 = round(model.coef_[3], 2)
beta_5 = round(model.coef_[4], 2)
beta_6 = round(model.coef_[5], 2)
beta_7 = round(model.coef_[6], 2)

print("Beta_0:", beta_0)
print("Beta for  central distance: ", beta_1)
print("Beta for sustainability: ", beta_2)
print("Beta for rating: ", beta_3)
print("Beta for comfort rating: ", beta_4)
print("Beta for review count: ", beta_5)
print("Betas for number of beds: ", beta_6)
print("Betas for dummy city: ", beta_7)

Beta_0: 1468.26
Beta for  central distance:  60.66
Beta for sustainability:  40.28
Beta for rating:  321.85
Beta for comfort rating:  77.32
Beta for review count:  -170.52
Betas for number of beds:  -17.21
Betas for dummy city:  165.98


In [162]:
# Third equation
data_c1_w1['event'] = 0
data_c1_w2['event'] = 1
data_c2_w1['event'] = 0
data_c2_w2['event'] = 1

data_eq3 = pd.concat([data_c1_w1, data_c2_w1, data_c2_w1, data_c2_w2]).drop(columns=['start_date', 'end_date', 'description'])
data_eq3['dummy_city'] = np.where(data_eq3['city'] == 'Barcelona',1 ,0)
data_eq3.drop(columns=['city'], inplace=True)
data_eq3 = data_eq3[data_eq3['price'] <= 1000000] #there is some real big outlier

data_eq3.head()

,central_distance,sustainability,rating,comfort_rating,review_count,price,num_beds,event,dummy_city
59,5.5,3.0,7.2,9.3,1047.0,581,2.0,0,1
3,5.5,1.0,8.3,9.3,3894.0,659,2.0,0,1
68,5.7,3.0,8.3,9.3,2879.0,713,1.0,0,1
186,2.6,3.0,8.0,9.3,1562.0,774,1.0,0,1
238,2.6,3.0,8.0,9.3,840.0,774,3.0,0,1


In [163]:
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

# Create a LinearRegression model
model = LinearRegression()
features = ['central_distance', 'sustainability', 'rating', 'comfort_rating',
    'review_count', 'num_beds', 'event', 'dummy_city']

# scale continuous variables
scaler = preprocessing.StandardScaler()
scaler.fit(data_eq3[features])## Fitted to the training set

X_data = scaler.transform(data_eq3[features])
X_data=pd.DataFrame(X_data, columns=features)

# Fit the model to the data_eq1
model.fit(X_data, data_eq3['price'])

# Extract the coefficients
beta_0 = round(model.intercept_, 2)
beta_1 = round(model.coef_[0], 2)
beta_2 = round(model.coef_[1], 2)
beta_3 = round(model.coef_[2], 2)
beta_4 = round(model.coef_[3], 2)
beta_5 = round(model.coef_[4], 2)
beta_6 = round(model.coef_[5], 2)
beta_7 = round(model.coef_[6], 2)
beta_8 = round(model.coef_[7], 2)

print("Beta_0:", beta_0)
print("Beta for  central distance: ", beta_1)
print("Beta for sustainability: ", beta_2)
print("Beta for rating: ", beta_3)
print("Beta for comfort rating: ", beta_4)
print("Beta for review count: ", beta_5)
print("Betas for number of beds: ", beta_6)
print("Betas for dummy event: ", beta_7)
print("Betas for dummy city: ", beta_8)

Beta_0: 1429.34
Beta for  central distance:  77.87
Beta for sustainability:  -11.75
Beta for rating:  339.92
Beta for comfort rating:  58.74
Beta for review count:  -176.15
Betas for number of beds:  -50.88
Betas for dummy event:  56.66
Betas for dummy city:  148.39


In [164]:
data_c1_w2.head()

,central_distance,sustainability,rating,comfort_rating,review_count,price,start_date,end_date,description,num_beds,city,event
305,5.7,3.0,8.3,9.4,2878.0,948,2024/06/17,2024/06/23,This modern hotel is located on the northern e...,1.0,Barcelona,1
326,4.5,3.0,8.2,9.4,2841.0,966,2024/06/17,2024/06/23,Hotel L’Alguer Nou is located just 1640 feet f...,1.0,Barcelona,1
16,1.6,3.0,7.6,9.4,3684.0,1008,2024/06/17,2024/06/23,Conveniently located less than 10 minutes’ wal...,1.0,Barcelona,1
108,2.9,1.0,7.3,9.4,9297.0,1025,2024/06/17,2024/06/23,This modern hotel has 250 spacious and fully e...,1.0,Barcelona,1
167,1.9,1.0,7.9,9.4,1884.0,1053,2024/06/17,2024/06/23,This modern hotel is located 215 yards from Ba...,1.0,Barcelona,1


In [165]:
from collections import Counter

# Room amenities vocabulary
Vocab1 = ['AC', 'air-conditioning', 'air conditioning']
Vocab2 = ['gym', 'pool', 'fitness']
Vocab3 = ['breakfast buffet', 'buffet breakfast']
Vocab4 = ['park', 'parking']

# Recreative amenities vocabulary


# Food amenities


r1 = re.compile("|".join(r"\b%s\b" % w for w in Vocab1))
r2 = re.compile("|".join(r"\b%s\b" % w for w in Vocab2))
r3 = re.compile("|".join(r"\b%s\b" % w for w in Vocab3))
r4 = re.compile("|".join(r"\b%s\b" % w for w in Vocab4))



In [166]:
data_eq4 = pd.concat([data_c1_w1, data_c2_w1, data_c2_w1, data_c2_w2]).drop(columns=['start_date', 'end_date'])
data_eq4['dummy_city'] = np.where(data_eq4['city'] == 'Barcelona',1 ,0)
data_eq4.drop(columns=['city'], inplace=True)
data_eq4 = data_eq4[data_eq4['price'] <= 50000] #there is some real big outlier

for r in [r1, r2, r3, r4]:
    Vocab_list=[]
    for s in data_eq4.description:
        wordcount = Counter(re.findall(r, s.lower()))
        #print(wordcount) # to check whether it runs well
        Vocab_list.append(sum(wordcount.values()))

    if r == r1:
        data_eq4['room_features'] = Vocab_list
        data_eq4['room_features'] = data_eq4['room_features']/data_eq4['room_features']
    elif r == r2:
        data_eq4['recreational_features'] = Vocab_list
        data_eq4['recreational_features'] = data_eq4['recreational_features']/data_eq4['recreational_features']
    elif r == r3:
        data_eq4['breakfast_features'] = Vocab_list
        data_eq4['breakfast_features'] = data_eq4['breakfast_features']/data_eq4['breakfast_features']
    else: 
        data_eq4['commodity_features'] = Vocab_list
        data_eq4['commodity_features'] = data_eq4['commodity_features']/data_eq4['commodity_features']

data_eq4.fillna(0, inplace=True)

In [175]:
data_eq4['dummy_features'] = np.where(data_eq4['room_features'] + data_eq4['recreational_features'] + data_eq4['breakfast_features'] + data_eq4['commodity_features'] >= 2, 1, 0)
print(data_eq4['dummy_features'].value_counts())
data_eq4.sort_values('dummy_features',  ascending=False).head(10)

dummy_features
0    972
1    569
Name: count, dtype: int64


,central_distance,sustainability,rating,comfort_rating,review_count,price,description,num_beds,event,dummy_city,room_features,recreational_features,breakfast_features,commodity_features,dummy_features
59,5.5,3.0,7.2,9.3,1047.0,581,This modern hotel offers free wired internet a...,2.0,0,1,0.0,0.0,1.0,1.0,1
201,2.3,3.0,8.3,9.3,6361.0,2233,"The Vincci Soma is located in central Madrid, ...",1.0,0,0,0.0,1.0,1.0,1.0,1
279,350.0,2.0,9.1,9.6,235.0,2302,"Hotel Montera Madrid, Curio Collection By Hilt...",1.0,0,0,1.0,1.0,0.0,0.0,1
62,1.7,2.0,8.9,9.7,1611.0,2399,Get the celebrity treatment with world-class s...,1.0,0,0,0.0,1.0,0.0,1.0,1
199,4.8,3.0,8.8,9.6,1509.0,2431,AC Hotel Cuzco is located on Madrid’s prestigi...,1.0,0,0,0.0,1.0,1.0,1.0,1
257,1.2,2.0,9.1,9.4,701.0,2684,"Set in a restored palace, URSO Hotel & Spa, a ...",1.0,0,0,1.0,1.0,0.0,1.0,1
224,1.2,3.0,8.3,9.8,66.0,2842,Get the celebrity treatment with world-class s...,1.0,0,0,0.0,1.0,0.0,1.0,1
321,2.0,3.0,8.9,9.9,234.0,3060,Get the celebrity treatment with world-class s...,1.0,0,0,0.0,1.0,0.0,1.0,1
170,2.1,2.0,8.9,9.7,586.0,3789,Get the celebrity treatment with world-class s...,1.0,0,0,0.0,1.0,1.0,1.0,1
288,350.0,3.0,9.4,9.6,795.0,3791,"Located in the Barrio de las Letras, Gran Hote...",1.0,0,0,0.0,0.0,1.0,1.0,1


In [178]:
data_eq5 = data_eq4[data_eq4['dummy_features'] == 1]

# Create a LinearRegression model
model = LinearRegression()
features = ['central_distance', 'sustainability', 'rating', 'comfort_rating',
    'review_count', 'num_beds', 'event', 'dummy_city', 'dummy_features']

# scale continuous variables
scaler = preprocessing.StandardScaler()
scaler.fit(data_eq4[features])## Fitted to the training set

X_data = scaler.transform(data_eq4[features])
X_data=pd.DataFrame(X_data, columns=features)

# Fit the model to the data_eq1
model.fit(X_data, data_eq4['price'])

# Extract the coefficients
beta_0 = round(model.intercept_, 2)
beta_1 = round(model.coef_[0], 2)
beta_2 = round(model.coef_[1], 2)
beta_3 = round(model.coef_[2], 2)
beta_4 = round(model.coef_[3], 2)
beta_5 = round(model.coef_[4], 2)
beta_6 = round(model.coef_[5], 2)
beta_7 = round(model.coef_[6], 2)
beta_8 = round(model.coef_[7], 2)
beta_9 = round(model.coef_[8], 2)

print("Beta_0:", beta_0)
print("Beta for  central distance: ", beta_1)
print("Beta for sustainability: ", beta_2)
print("Beta for rating: ", beta_3)
print("Beta for comfort rating: ", beta_4)
print("Beta for review count: ", beta_5)
print("Betas for number of beds: ", beta_6)
print("Betas for dummy event: ", beta_7)
print("Betas for dummy city: ", beta_8)
print("Betas for dummy features: ", beta_9)

Beta_0: 1429.34
Beta for  central distance:  83.09
Beta for sustainability:  -13.81
Beta for rating:  335.73
Beta for comfort rating:  58.23
Beta for review count:  -176.71
Betas for number of beds:  -46.89
Betas for dummy event:  56.63
Betas for dummy city:  150.43
Betas for dummy features:  58.07
